In [ ]:
import pandas
import matplotlib.pyplot as plt

enade2017=pandas.read_csv("datasets\MICRODADOS_ENADE_2017.txt", sep=';',dtype={"DS_VT_ESC_OFG": str, 
                                                                               'DS_VT_ESC_OCE':str,
                                                                              'DS_VT_ACE_OCE':str,
                                                                              'NT_GER':str,
                                                                              'NT_FG':str,
                                                                              'NT_OBJ_FG':str,
                                                                              'NT_DIS_FG':str,
                                                                              'NT_CE':str,
                                                                              'NT_OBJ_CE':str,
                                                                              'NT_DIS_CE':str})

In [ ]:
enade2017.columns[0:10]

In [ ]:
'''
nota geral: 0-98 
questão08: Qual a renda total de sua família, incluindo seus rendimentos?
A = Até 1,5 salário mínimo (até R$ 1.405,50).
B = De 1,5 a 3 salários mínimos (R$ 1.405,51 a R$ 2.811,00).
C = De 3 a 4,5 salários mínimos (R$ 2.811,01 a R$ 4.216,50).
D = De 4,5 a 6 salários mínimos (R$ 4.216,51 a R$ 5.622,00).
E = De 6 a 10 salários mínimos (R$ 5. 622,01 a R$ 9.370,00).
F = De 10 a 30 salários mínimos (R$ 9.370,01 a R$ 28.110,00).
G = Acima de 30 salários mínimos (mais de R$ 28.110,00).

i05 escolaridade da mae
A = Nenhuma.
B = Ensino fundamental: 1º ao 5º ano (1ª a 4ª série).
C = Ensino fundamental: 6º ao 9º ano (5ª a 8ª série).
D = Ensino médio.
E = Ensino Superior - Graduação.
F = Pós-graduação.

'''

tabela = pandas.DataFrame(enade2017, columns=['NT_GER', 'CO_GRUPO', 'QE_I08', 'CO_IES', 'QE_I05'])
tabela.head(10)

In [ ]:
##limpeza dos dados

#substitui vírgula por ponto
tabela['NT_GER'] = tabela['NT_GER'].str.replace(',', '.')
print (tabela['NT_GER'])

#observe os NaN (not a number)

In [ ]:
'''No arquivo 'Dicionário de variáveis dos Microdados do Enade_Edição 2017' descreve que:
o codigo 222 no campo TP_PR_GER significa ausente 
556 e 888 são participações desconsideradas.
Portanto, algumas notas podem ser desconsideradas dependendo do seu objetivo, 
e aqui, será de calcular a média daqueles que fizeram a prova.
'''
tabela=tabela.loc[(tabela['NT_GER'].notnull())]
#print(tabela['NT_GER'])
#converte de str para float
tabela['NT_GER'] = pandas.to_numeric(tabela['NT_GER'])
print(tabela['NT_GER'])
print(tabela['NT_GER'].mean())

In [ ]:
tabela['NT_GER'].describe()

In [ ]:
#outros comandos
print('indice da primeira maior nota: ', tabela['NT_GER'].idxmax())
print('Maior nota: ', tabela['NT_GER'][72257])
#print(tabela['NT_GER'].idxmax())

In [ ]:
#Calcula a média de um curso especifico
#Código da área de enquadramento do curso no Enade == ciencia da computacao

ccomp = tabela[tabela['CO_GRUPO']==4004]

print(ccomp)

In [ ]:
ccomp['NT_GER'].describe()

In [ ]:
#do curso do IFNMG
ifccomp = ccomp[ccomp['CO_IES']==3188]
ifccomp.describe()

In [ ]:
#somente as notas de quem respondeu a questão sobre a renda
ccomp=ccomp.loc[(ccomp['QE_I08'].notnull())]
ccomp.NT_GER.describe()

In [ ]:
ccomp=ccomp.loc[(ccomp['QE_I05'].notnull())]
ccomp.NT_GER.describe()

In [ ]:
ccomp.QE_I08.head(10)

ccomp['QE_I08'] = ccomp['QE_I08'].map({'A': 1, 'B': 2, 'C': 3, 'D': 4,'E': 5, 'F':6,'G':7})

ccomp.QE_I08.head(10)

In [ ]:
import matplotlib.pyplot as plt
#visualmente
plt.scatter( ccomp.NT_GER, ccomp.QE_I08)
plt.ylabel('Faixa de renda')
plt.xlabel('Nota do curso de C. da Comp.')
plt.show()

In [ ]:
ccomp.QE_I05 = ccomp['QE_I05'].map({'A': 1, 'B': 2, 'C': 3, 'D': 4,'E': 5, 'F':6})

ccomp.QE_I05.head(10)

In [ ]:
#visualmente
plt.scatter( ccomp.NT_GER, ccomp.QE_I05)
plt.ylabel('Escolaridade da mãe')
plt.xlabel('Nota do curso de C. da Comp.')
plt.show()

In [ ]:
escolaridade = ccomp.loc[ccomp.QE_I05 ==1]
escolaridade.NT_GER.describe()

In [ ]:
#outros exemplos de consultas
tabela = pandas.DataFrame(enade2017, columns=['NT_GER', 'CO_CATEGAD','CO_GRUPO', 'QE_I08', 'CO_IES', 'QE_I05'])

#apenas IES federais
ccompfederal = tabela.loc[ (tabela.CO_CATEGAD==1)]
ccompfederal.head()

In [ ]:
#Como verificar se a média dos alunos do campo QE_I05 para todos os alunos são realmente diferentes
qe05 = pandas.DataFrame(tabela, columns=['NT_GER', 'QE_I05'])
qe05.boxplot(by='QE_I05')

In [ ]:
#para os alunos de computacao
qe05 = pandas.DataFrame(ccomp, columns=['NT_GER', 'QE_I05'])
qe05.boxplot(by='QE_I05')

In [ ]:
#A análise de variância (ANOVA) é um teste usado para comparar as médias de dois grupos. 
#suposições: normalidade, independência e variância homegênea

#nota geral agrupada pela questão 05
tabela['NT_GER'].groupby(tabela['QE_I05']).describe()

In [ ]:
#teste das suposições
import scipy.stats as stats

print(stats.shapiro(qe05.NT_GER.loc[qe05.QE_I05 ==1]))

print(stats.shapiro(qe05.NT_GER.loc[qe05.QE_I05 ==6]))

#a saída do teste é o valor W e o p-value
#Considera-se que a suposição não é violada quando o p-value é maior que alpha, normalmente 0.05

#pelo histograma, também é possível observar a distribuição dos dados. 
qe05.NT_GER.loc[qe05.QE_I05 ==6].hist()

In [ ]:
# teste não paramétrico são métodos que não assumem uma distribuição específica para os dados.
stat, p = stats.mannwhitneyu(qe05.NT_GER.loc[qe05.QE_I05 ==1], qe05.NT_GER.loc[qe05.QE_I05 ==6])

print('Mann-Whitney: Estatisticas=%.3f, p=%.3f' % (stat, p))

alpha = 0.05
if p > alpha:
	print('Mesma distribuição')
else:
	print('Distribução diferente')
    
    
stat, p = stats.kruskal(qe05.NT_GER.loc[qe05.QE_I05 ==1], qe05.NT_GER.loc[qe05.QE_I05 ==6])
print('Kruskal-Wallis: Estatisticas=%.3f, p=%.3f' % (stat, p))

if p > alpha:
	print('Mesma distribuição')
else:
	print('Distribução diferente')
    
#anova
stat, p = stats.f_oneway(qe05.NT_GER.loc[qe05.QE_I05 ==1], qe05.NT_GER.loc[qe05.QE_I05 ==6])
print('Anova: Estatisticas=%.3f, p=%.3f' % (stat, p))
if p > alpha:
	print('Mesma distribuição')
else:
	print('Distribução diferente')